In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, LSTM
from tensorflow.keras import datasets
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras import Model

(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

In [2]:
x_train.shape

(60000, 28, 28)

In [12]:
# MNIST数据集使用lstm进行预测试，可以理解为一张图片即为28*28的数据。单张图片有28条（行）数据，每条的样本为28（列）的长度。
timesteps = 28
data_dim = 28

In [24]:
model1 = Sequential([
    LSTM(32, return_sequences=True,  input_shape=(timesteps, data_dim)),
    LSTM(16),
    Dense(32, 'relu'),
    Dense(10, 'softmax')
])

In [15]:
inputss = Input(shape=(timesteps, data_dim))
x = LSTM(32, return_sequences=True)(inputss)
x = LSTM(16)(x)
x = Dense(32, 'relu')(x)
outputs = Dense(10, 'softmax')(x)

model2 = Model(inputs=inputss, outputs=outputs)

In [19]:
# 使用 model subclassing 方法搭建网络后，提示不支持 unit8 的数据格式，需要进行类型转换才能正常运行。
x_train = tf.cast(x_train,tf.float32)
x_test = tf.cast(x_test,tf.float32)

In [22]:
class MnistLSTM(tf.keras.Model):
    def __init__(self):
        super().__init__()
        # 无论是sequential还是model subclassing， 用参数说明数据的输入形状都不是必须的，input_shape可要可不要
        self.lstm1 = LSTM(32, return_sequences=True, input_shape=(28, 28))
        self.lstm2 = LSTM(16)
        self.dense1 = Dense(32, 'relu')
        self.dense2 = Dense(10, 'softmax')
    
    def call(self, inputs):
        x = self.lstm1(inputs)
        x = self.lstm2(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x
    
    
model3 = MnistLSTM()

In [25]:
model_lst = [model1]
for i, model in enumerate(model_lst):
    print('-'*20)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(x=x_train, y=y_train, epochs=5)
    model.evaluate(x_test, y_test)

--------------------
Epoch 1/5
1875/1875 [==============================] - 35s 19ms/step - loss: 0.6890 - accuracy: 0.7682
Epoch 2/5
1875/1875 [==============================] - 35s 18ms/step - loss: 0.2999 - accuracy: 0.9057
Epoch 3/5
1875/1875 [==============================] - 34s 18ms/step - loss: 0.2146 - accuracy: 0.9342
Epoch 4/5
1875/1875 [==============================] - 34s 18ms/step - loss: 0.1690 - accuracy: 0.9473
Epoch 5/5
313/313 [==============================] - 3s 10ms/step - loss: 0.1399 - accuracy: 0.9554
